In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import keras_tuner as kt
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

C:\Users\Rafi\AppData\Local\Temp\ipykernel_7612\291211984.py:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [2]:
# Load and preprocess the data
data = pd.read_csv('nutrition.csv')
data.rename(columns={'name': 'label'}, inplace=True)

# Extract features and labels
x = data[['calories', 'proteins', 'fat', 'carbohydrate']]
y = data['label']

# Normalize the features
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

# Encode the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((x_scaled, y_encoded)).batch(32)


In [4]:
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units=hp.Int('units1', min_value=32, max_value=256, step=32),
                                    activation='relu', input_shape=(x_scaled.shape[1],)))
    model.add(tf.keras.layers.Dense(units=hp.Int('units2', min_value=32, max_value=256, step=32),
                                    activation='relu'))
    model.add(tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax'))

    model.compile(optimizer=hp.Choice('optimizer', ['adam', 'rmsprop']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Hyperparameter Tuning

In [7]:
tuner = RandomSearch(
    build_model,
    objective='accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='my_tuning_dir',
    project_name='nutrition_model_tuning')

tuner.search(x=x_scaled, y=y_encoded, epochs=1000)
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 5 Complete [00h 04m 26s]
accuracy: 0.7852897644042969

Best accuracy So Far: 0.908618152141571
Total elapsed time: 00h 20m 27s


In [9]:
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(train_dataset, epochs=1000)

Epoch 1/1000
43/43 [==============================] - 1s 6ms/step - loss: 7.2367 - accuracy: 0.0000e+00
Epoch 2/1000
43/43 [==============================] - 0s 6ms/step - loss: 7.1208 - accuracy: 0.0074
Epoch 3/1000
43/43 [==============================] - 0s 6ms/step - loss: 6.5832 - accuracy: 0.0015
Epoch 4/1000
43/43 [==============================] - 0s 6ms/step - loss: 5.9267 - accuracy: 0.0037
Epoch 5/1000
43/43 [==============================] - 0s 6ms/step - loss: 5.5510 - accuracy: 0.0067
Epoch 6/1000
43/43 [==============================] - 0s 6ms/step - loss: 5.2778 - accuracy: 0.0097
Epoch 7/1000
43/43 [==============================] - 0s 6ms/step - loss: 5.0403 - accuracy: 0.0163
Epoch 8/1000
43/43 [==============================] - 0s 6ms/step - loss: 4.8578 - accuracy: 0.0171
Epoch 9/1000
43/43 [==============================] - 0s 8ms/step - loss: 4.6963 - accuracy: 0.0208
Epoch 10/1000
43/43 [==============================] - 0s 6ms/step - loss: 4.5724 - accuracy: 0.

In [10]:
# Function to predict the closest food
def predict_food(calories, proteins, fat, carbohydrate):
    input_data = np.array([[calories, proteins, fat, carbohydrate]])
    input_data_scaled = scaler.transform(input_data)
    predictions = hypermodel.predict(input_data_scaled)
    predicted_label = label_encoder.inverse_transform([np.argmax(predictions)])
    return predicted_label[0]

# Test the function with the provided example
input_nutrients = [25, 1.6, 0.2, 5.3]
predicted_food = predict_food(*input_nutrients)
print(f"Input nutrients: {input_nutrients} -> Predicted food: {predicted_food}")

C:\Users\Rafi\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 123ms/step
Input nutrients: [25, 1.6, 0.2, 5.3] -> Predicted food: Kool Merah kool putih


In [11]:
file_name = 'model.h5'
keras.models.save_model(hypermodel, file_name)

C:\Users\Rafi\AppData\Local\Temp\ipykernel_10816\2707932765.py:2: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  keras.models.save_model(model, file_name)


In [12]:
from keras.models import load_model
save_model = load_model('model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(save_model)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Rafi\AppData\Local\Temp\tmpqp54dj_l\assets


INFO:tensorflow:Assets written to: C:\Users\Rafi\AppData\Local\Temp\tmpqp54dj_l\assets


370004